**Описание датасета**

* pid: Numeric - ID покемона
* HP: Numeric - Очки здоровья
* Attack: Numeric - Сила обычной атаки
* Defense: Numeric - Сила обычной защиты
* Sp. Atk: Numeric - Сила специальной атаки
* Sp. Def: Numeric - Сила специальной защиты
* Speed: Numeric - Скорость движений
* Legendary: Boolean - «True», если покемон редкий
* Class 1: Categorical - Класс покемона
* Class 2: Categorical - Класс покемона

In [27]:
import warnings
# Отключение предупреждений (warnings)
warnings.filterwarnings("ignore")

import pandas as pd

from scipy.stats import ttest_ind
from scipy.stats import f_oneway, shapiro

pokemon = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/pokemon.csv', on_bad_lines='skip')  # Откроем датасет
pokemon.head()

# Обратите внимание, что у покемона может быть один или два класса.
# Если у покемона два класса, считается, что они имеют одинаковую значимость.

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [2]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pid        800 non-null    int64 
 1   Name       799 non-null    object
 2   Class 1    800 non-null    object
 3   Class 2    414 non-null    object
 4   HP         800 non-null    int64 
 5   Attack     800 non-null    int64 
 6   Defense    800 non-null    int64 
 7   Sp. Atk    800 non-null    int64 
 8   Sp. Def    800 non-null    int64 
 9   Speed      800 non-null    int64 
 10  Legendary  800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 63.4+ KB


In [3]:
pokemon.isna().sum()

,0
pid,0
Name,1
Class 1,0
Class 2,386
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0
Speed,0


**Задание № 1:**
Профессор Оук подозревает, что покемоны в классе Grass имеют более сильную обычную атаку, чем покемоны в классе Rock. Проверьте, прав ли он, и убедите его в своём выводе статистически.

Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;

Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

In [4]:
grass_pokemon = pokemon[(pokemon['Class 1'] == 'Grass') & (pokemon['Class 2'] != 'Rock')]
rock_pokemon = pokemon[(pokemon['Class 1'] == 'Rock') & (pokemon['Class 2'] != 'Grass')]

In [5]:
grass_attack = grass_pokemon['Attack']
rock_attack = rock_pokemon['Attack']

In [24]:
t_stat, p_value = ttest_ind(grass_attack, rock_attack, alternative='greater', equal_var=True)
print("t-статистика:", t_stat)
print("p-value:", p_value)
if p_value < 0.05:
    print("Отклоняем нулевую гипотезу: средняя атака Grass статистически значимо выше, чем у Rock.")
else:
    print("Не отклоняем нулевую гипотезу: недостаточно доказательств, что атака Grass выше, чем у Rock.")


t-статистика: -3.6867299152069792
p-value: 0.9998227575495493
Не отклоняем нулевую гипотезу: недостаточно доказательств, что атака Grass выше, чем у Rock.


In [ ]:
# t-статистика: указывает на то, что Grass-покемоны значительно уступают Rock-покемонам по средней атаке.

**Задание № 2:**
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос, а правда ли, что покемоны в классе Water в среднем быстрее, чем покемоны в классе Normal.

Проверьте, прав ли он, и убедите его в своём выводе статистически.

Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;

Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

In [23]:
water_pokemon = pokemon[(pokemon['Class 1'] == 'Water') & (pokemon['Class 2'] != 'Normal')]
normal_pokemon = pokemon[(pokemon['Class 1'] == 'Normal') & (pokemon['Class 2'] != 'Water')]

water_speed = water_pokemon['Speed']
normal_speed = normal_pokemon['Speed']

t_stat, p_value = ttest_ind(water_speed, normal_speed, equal_var=False, alternative='greater')
print("t-статистика:", t_stat)
print("p-value:", p_value)
if p_value < 0.05:
    print("Отклоняем нулевую гипотезу: покемоны Water статистически значимо быстрее, чем Normal.")
else:
    print("Не отклоняем нулевую гипотезу: недостаточно доказательств, что Water быстрее Normal.")

t-статистика: -1.542980807203786
p-value: 0.9377234939112589
Не отклоняем нулевую гипотезу: недостаточно доказательств, что Water быстрее Normal.


In [ ]:
#Принимая во внимание отрицательную величину t_stat очень вероятно, что напротив средняя скорость Normal превосходит Water.

**Задание № 3:**
Профессор Оук тот еще безумец. Он изобрёл сыворотку, способную ускорить покемона. Однако мы усомнились в эффективности его вакцины. Професоор дал эту сыворотку следующим покемонам: смотри массив treathed_pokemon. Проверьте, работает ли вообще его сыворотка, убедите всех в своём выводе статистически.

Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

In [28]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill', 'Mega Alakazam',
                    'Deoxys Normal Forme', 'Mega Lopunny']

In [29]:
filtered_pokemon = pokemon[pokemon['Name'].str.contains('Beedrill|Alakazam|Deoxy|Lopunny', case=False, na=False)]

print(filtered_pokemon[['Name', 'Speed']])

                     Name  Speed
18               Beedrill     75
19          Mega Beedrill    145
70               Alakazam    120
71          Mega Alakazam    150
428   Deoxys Normal Forme    150
429    DeoxysAttack Forme    150
430  Deoxys Defense Forme     90
431    Deoxys Speed Forme    180
475               Lopunny    105
476          Mega Lopunny    135


In [30]:
standart_pokemon = ['Beedrill', 'Alakazam', 'DeoxysAttack Forme', 'Deoxys Speed Forme', 'Deoxys Defense Forme', 'Lopunny']

In [31]:
standart_speed = pokemon[pokemon['Name'].isin(standart_pokemon)]['Speed']
treathed_speed = pokemon[pokemon['Name'].isin(treathed_pokemon)]['Speed']

In [34]:
t_stat, p_value = ttest_ind(standart_speed, treathed_speed, equal_var=True)
print("p-value:", p_value)

if p_value < 0.05:
    print("Средние значения статистически различаются — есть эффект! Покемоны стали быстрее.")
else:
    print(" Статистически значимой разницы не выявлено — сыворотка, возможно, неэффективна.")

p-value: 0.24998980625876047
 Статистически значимой разницы не выявлено — сыворотка, возможно, неэффективна.


**Задание № 4:**
Профессор Оук всегда любил истории про легендарных покемонов. Однако профессор не очень уверен, что они лучше остальных покемонов. Оук предложил разобраться в этом нам. Проверьте, действительно ли сумма характеристик HP,Attack,Defense у легендарных покемонов выше, чем у других покемонов?

А произведение этих же параметров?

Найдите ответы на эти вопросы и убедите всех в своём выводе статистически.

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

In [36]:
legendary_pokemon = pokemon[pokemon['Legendary'] == True]
other_pokemon = pokemon[pokemon['Legendary'] == False]

In [37]:
legendary_sums = legendary_pokemon[['HP', 'Attack', 'Defense']].sum(axis=1)
other_sums = other_pokemon[['HP', 'Attack', 'Defense']].sum(axis=1)


In [38]:
sum_t_stat, sum_p_value = ttest_ind(legendary_sums, other_sums, equal_var=False)
print(f"Sum P-value: {sum_p_value:.10f}")
if sum_p_value < 0.05:
    print("Легендарные покемоны действительно имеют более высокую сумму характеристик.")
else:
    print("Разница в сумме характеристик статистически незначима.")

Sum P-value: 0.0000000000
Легендарные покемоны действительно имеют более высокую сумму характеристик.


In [39]:
legendary_products = legendary_pokemon[['HP', 'Attack', 'Defense']].prod(axis=1)
other_products = other_pokemon[['HP', 'Attack', 'Defense']].prod(axis=1)

In [40]:
prod_t_stat, prod_p_value = ttest_ind(legendary_products, other_products, equal_var=False)
print(f"Product P-value: {prod_p_value:.10f}")
if prod_p_value < 0.05:
    print("Легендарные покемоны действительно имеют более высокое произведение характеристик.")
else:
    print("Разница в произведении характеристик статистически незначима.")

Product P-value: 0.0000000000
Легендарные покемоны действительно имеют более высокое произведение характеристик.


**Задание № 5:**
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боёв Оук выделил четыре класса best_defence_class, которые на его взгляд одинаковы по "силе обычной защиты" Defense.

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Всё та же статистика вам в помощь!

Вы можете предположить, что распределение параметров защитных характеристик является нормальным для всех классов покемонов.

In [ ]:
best_defence_class = ['Rock', 'Ground', 'Steel', 'Ice']

In [42]:
rock_defense = pokemon[pokemon['Class 1'] == 'Rock']['Defense']
ground_defense = pokemon[pokemon['Class 1'] == 'Ground']['Defense']
steel_defense = pokemon[pokemon['Class 1'] == 'Steel']['Defense']
ice_defense = pokemon[pokemon['Class 1'] == 'Ice']['Defense']

# ANOVA-тест
f_stat, p_value = f_oneway(rock_defense, ground_defense, steel_defense, ice_defense)

print("p-value:", p_value)
if p_value < 0.05:
    print("Гипотеза отклоняется: классы покемонов статистически отличаются по уровню защиты.")
else:
    print("Гипотеза подтверждается: различия в уровне защиты между классами не являются статистически значимыми.")

p-value: 3.0803014649204294e-06
Гипотеза отклоняется: классы покемонов статистически отличаются по уровню защиты.
